In [3]:
# Download data

import os
import urllib.request
import gzip
import shutil

url = 'https://storage.googleapis.com/gresearch/wit/wit_v1.train.all-1percent_sample.tsv.gz'
filename = 'data/wit/data.tsv'

if os.path.exists(filename):
    print("The file exists")

else:
    # Download the data from the URL
    with urllib.request.urlopen(url) as response:
      with open(filename + '.gz', 'wb') as f:
        f.write(response.read())
    
    # Extract the data from the compressed file
    with gzip.open(filename + '.gz', 'rb') as f_in:
      with open(filename, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

    print("The file was downloaded")

The file exists


In [8]:
# Create Postgres table with initial data

import os
import psycopg2

# Get the DB connection string from the environment variable
db_connection_string = os.environ.get('DATABASE_URL')

# Check if the variable exists
if db_connection_string is not None:
    conn = psycopg2.connect(db_connection_string)
    cursor = conn.cursor()

    with open('data/wit/create_table.sql', 'r') as sql_file:
        sql_script = sql_file.read()
    cursor.execute(sql_script)
    print("Create table")

    count_query = f"SELECT COUNT(*) FROM tsv_data"
    cursor.execute(count_query)
    row_count = cursor.fetchone()[0]

    if row_count == 0:
        with open('data/wit/copy_data.sql', 'r') as sql_file:
            sql_script = sql_file.read()
        cursor.execute(sql_script)
        print("Copied data")
    else:
        print("No need to copy data")

    conn.commit()

    cursor.close()
    conn.close()

    print("Completed")
else:
    print("DB_CONNECTION_STRING environment variable is not set.")

Create table
No need to copy data
Completed
